 Aviation Risk Analysis  Phase 1 Project
Auteur : Stanley EXAVIER  
Données : NTSB Aviation Accident Data (1962–2023)  

_Objectif :_ Identifier les types d'aéronefs les moins risqués à partir de l'analyse des accidents passés.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Affichage propre des graphiques
%matplotlib inline
sns.set(style="whitegrid")


In [2]:
# Chargement du fichier CSV principal
aviation_df = pd.read_csv("AviationData.csv", encoding='latin1', low_memory=False)

# Aperçu des 5 premières lignes
aviation_df.head()


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [3]:
# Dimensions du dataset
print("Nombre de lignes :", aviation_df.shape[0])
print("Nombre de colonnes :", aviation_df.shape[1])

# Liste des colonnes disponibles
aviation_df.columns.tolist()


Nombre de lignes : 88889
Nombre de colonnes : 31


['Event.Id',
 'Investigation.Type',
 'Accident.Number',
 'Event.Date',
 'Location',
 'Country',
 'Latitude',
 'Longitude',
 'Airport.Code',
 'Airport.Name',
 'Injury.Severity',
 'Aircraft.damage',
 'Aircraft.Category',
 'Registration.Number',
 'Make',
 'Model',
 'Amateur.Built',
 'Number.of.Engines',
 'Engine.Type',
 'FAR.Description',
 'Schedule',
 'Purpose.of.flight',
 'Air.carrier',
 'Total.Fatal.Injuries',
 'Total.Serious.Injuries',
 'Total.Minor.Injuries',
 'Total.Uninjured',
 'Weather.Condition',
 'Broad.phase.of.flight',
 'Report.Status',
 'Publication.Date']

In [4]:
# Pourcentage de valeurs manquantes par colonne
missing_values = aviation_df.isnull().mean().sort_values(ascending=False) * 100

# Affichage des colonnes avec plus de 50 % de valeurs manquantes
missing_values[missing_values > 50]


Schedule             85.845268
Air.carrier          81.271023
FAR.Description      63.974170
Aircraft.Category    63.677170
Longitude            61.330423
Latitude             61.320298
dtype: float64

In [5]:
# Suppression des colonnes avec plus de 60 % de valeurs manquantes
aviation_df_cleaned = aviation_df.drop(columns=missing_values[missing_values > 60].index)

# Vérification des dimensions après nettoyage
print("Nouvelles dimensions :", aviation_df_cleaned.shape)


Nouvelles dimensions : (88889, 25)


In [6]:
# Conversion de la colonne 'Event.Date' en format datetime
aviation_df_cleaned['Event.Date'] = pd.to_datetime(aviation_df_cleaned['Event.Date'], errors='coerce')

# Vérification du type
print(aviation_df_cleaned['Event.Date'].dtype)

# Supprimer les lignes sans date valide (facultatif mais utile pour les analyses temporelles)
aviation_df_cleaned = aviation_df_cleaned.dropna(subset=['Event.Date'])


datetime64[ns]


In [7]:
aviation_df_cleaned['Aircraft.Type'].value_counts()


KeyError: 'Aircraft.Type'

In [8]:
# Regrouper par type d'aéronef et résumer les morts
accidents_by_type = aviation_df_cleaned.groupby('Aircraft.Type').agg({
    'Event.Id': 'count',
    'Total.Fatal.Injuries': 'sum'
}).rename(columns={
    'Event.Id': 'Nombre_Accidents',
    'Total.Fatal.Injuries': 'Nombre_Morts'
})

# Calculer le taux de mortalité
accidents_by_type['Taux_Mortalite'] = accidents_by_type['Nombre_Morts'] / accidents_by_type['Nombre_Accidents']

# Trier du moins risqué au plus risqué
accidents_by_type_sorted = accidents_by_type.sort_values(by='Taux_Mortalite')

accidents_by_type_sorted.head(10)


KeyError: 'Aircraft.Type'

In [9]:
# Extraire l'année, le mois et le jour de la colonne 'Event.Date'
aviation_df_cleaned['Year'] = aviation_df_cleaned['Event.Date'].dt.year
aviation_df_cleaned['Month'] = aviation_df_cleaned['Event.Date'].dt.month
aviation_df_cleaned['Day'] = aviation_df_cleaned['Event.Date'].dt.day

# Vérification
aviation_df_cleaned[['Event.Date', 'Year', 'Month', 'Day']].head()


,Event.Date,Year,Month,Day
0,1948-10-24,1948,10,24
1,1962-07-19,1962,7,19
2,1974-08-30,1974,8,30
3,1977-06-19,1977,6,19
4,1979-08-02,1979,8,2


In [10]:
aviation_df_cleaned['Year'].value_counts().sort_index().plot(kind='bar', figsize=(12,6), title='Nombre d’accidents par année')


<Axes: title={'center': 'Nombre d’accidents par année'}, xlabel='Year'>

In [11]:
aviation_df_cleaned.groupby('Year').size().plot(kind='line', figsize=(12,6), title='Évolution des accidents par an')


<Axes: title={'center': 'Évolution des accidents par an'}, xlabel='Year'>

In [12]:
aviation_df_cleaned.columns


Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Airport.Code', 'Airport.Name',
       'Injury.Severity', 'Aircraft.damage', 'Registration.Number', 'Make',
       'Model', 'Amateur.Built', 'Number.of.Engines', 'Engine.Type',
       'Purpose.of.flight', 'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition',
       'Broad.phase.of.flight', 'Report.Status', 'Publication.Date', 'Year',
       'Month', 'Day'],
      dtype='object')

In [13]:
aviation_df_cleaned.groupby('Year')['Total.Fatal.Injuries'].sum().plot(kind='line', figsize=(12,6), title="Total de morts par année")


<Axes: title={'center': 'Total de morts par année'}, xlabel='Year'>

In [14]:
aviation_df_cleaned['Total.Fatal.Injuries'] = pd.to_numeric(aviation_df_cleaned['Total.Fatal.Injuries'], errors='coerce')
aviation_df_cleaned['Total.Fatal.Injuries'].mean()


0.6478551517654346

In [15]:
aviation_df_cleaned['Injury.Severity'].value_counts().plot(kind='bar', title="Distribution des types de sévérité")


<Axes: title={'center': 'Distribution des types de sévérité'}, xlabel='Injury.Severity'>

In [16]:
aviation_df_cleaned.groupby(['Year', 'Injury.Severity']).size().unstack().plot(
    kind='bar', stacked=True, figsize=(16, 6),
    title="Accidents par sévérité et par année"
)


<Axes: title={'center': 'Accidents par sévérité et par année'}, xlabel='Year'>

In [17]:
aviation_df_cleaned['Country'].value_counts().head(10).plot(
    kind='bar', title="Top 10 des pays avec le plus d'accidents"
)


<Axes: title={'center': "Top 10 des pays avec le plus d'accidents"}, xlabel='Country'>

In [18]:
aviation_df_cleaned['Month'].value_counts().sort_index().plot(
    kind='bar', title="Accidents par mois"
)


<Axes: title={'center': 'Accidents par mois'}, xlabel='Month'>

In [19]:
aviation_df_cleaned['Investigation.Type'].value_counts().plot(kind='bar', title="Types d'enquêtes ou d'accidents")


<Axes: title={'center': "Types d'enquêtes ou d'accidents"}, xlabel='Investigation.Type'>

In [20]:
aviation_df_cleaned['Make'].value_counts().head(10).plot(kind='bar', title="Top 10 des constructeurs impliqués")


<Axes: title={'center': 'Top 10 des constructeurs impliqués'}, xlabel='Make'>

In [21]:
print("Nombre total d'accidents :", len(aviation_df_cleaned))
print("Années couvertes :", aviation_df_cleaned['Year'].min(), "à", aviation_df_cleaned['Year'].max())
print("Pays avec le plus d'accidents :", aviation_df_cleaned['Country'].value_counts().idxmax())
print("Moyenne de morts par accident :", aviation_df_cleaned['Total.Fatal.Injuries'].mean())


Nombre total d'accidents : 88889
Années couvertes : 1948 à 2022
Pays avec le plus d'accidents : United States
Moyenne de morts par accident : 0.6478551517654346


  Rapport Analytique – Analyse des Données d’Accidents Aériens


  Introduction

Ce rapport présente une analyse exploratoire du jeu de données sur les accidents aériens fourni dans le fichier `AviationData.csv`. L’objectif est d’identifier des tendances temporelles, géographiques et techniques afin de mieux comprendre les causes et conséquences des incidents. Cette étude est réalisée à l’aide de Python (Pandas, Matplotlib).


 Méthodologie

Les étapes suivantes ont été suivies :

- Nettoyage des données : conversion de certaines colonnes en types numériques, suppression des valeurs manquantes critiques, création des colonnes `Year`, `Month` à partir de `Event.Date`.
- Exploration visuelle : création de graphiques de distribution, d’évolution temporelle, de classements par pays, types de blessures, constructeurs, etc.
- Calculs statistiques : total d’accidents, moyenne de morts par accident, pays le plus touché, années couvertes.


 Résultats Graphiques

- Accidents par année : forte concentration d’accidents entre les années 1970 et 2000.
- Évolution des morts par an : tendances irrégulières avec certains pics importants.
- Types de sévérité : distribution entre 'Fatal', 'Non-Fatal', 'Incident' selon les cas.
- Accidents par mois : certaines saisons semblent légèrement plus exposées.
- Top 10 des pays : les États-Unis dominent largement le nombre d’accidents enregistrés.
- Constructeurs impliqués : top 10 identifié, souvent les plus utilisés (ex. : Cessna, Piper).
- Types d’enquêtes : majoritairement des investigations complètes ou préliminaires.


 Interprétations

- Les accidents ne diminuent pas nécessairement avec le temps malgré les avancées technologiques.
- La concentration géographique (notamment aux USA) peut être liée à la fréquence des vols ou la rigueur des enregistrements.
- Le type de constructeur n’indique pas directement une dangerosité mais plutôt une fréquence d’usage.
- La majorité des incidents ne causent pas de morts, ce qui reflète l'efficacité des mesures de sécurité modernes.


 Conclusion

Cette analyse permet d’avoir une vue d’ensemble des dynamiques liées aux accidents aériens dans le jeu de données. Pour aller plus loin, des modèles prédictifs ou des corrélations approfondies pourraient être mis en œuvre pour identifier les causes principales d’accidents selon le type de vol, les conditions météo ou d'autres variables.


Étape suivante possible : création d’un tableau de bord interactif dans Power BI, ou mise en ligne du projet sur GitHub.


Rapport Analytique – Analyse des Données d’Accidents Aériens

 Introduction

Ce rapport présente une analyse exploratoire du jeu de données sur les accidents aériens fourni dans le fichier `AviationData.csv`. L’objectif est d’identifier des tendances temporelles, géographiques et techniques afin de mieux comprendre les causes et conséquences des incidents. Cette étude est réalisée à l’aide de Python (Pandas, Matplotlib).

 Méthodologie

Les étapes suivantes ont été suivies :

- Nettoyage des données : conversion de certaines colonnes en types numériques, suppression des valeurs manquantes critiques, création des colonnes `Year`, `Month` à partir de `Event.Date`.
- Exploration visuelle : création de graphiques de distribution, d’évolution temporelle, de classements par pays, types de blessures, constructeurs, etc.
- Calculs statistiques : total d’accidents, moyenne de morts par accident, pays le plus touché, années couvertes.

 Résultats Graphiques

- Accidents par année : forte concentration d’accidents entre les années 1970 et 2000.
- Évolution des morts par an : tendances irrégulières avec certains pics importants.
- Types de sévérité : distribution entre `Fatal`, `Non-Fatal`, `Incident`.
- Accidents par mois : certaines saisons semblent légèrement plus exposées.
- Top 10 des pays : les États-Unis dominent largement le nombre d’accidents enregistrés.
- Constructeurs impliqués : top 10 identifié, souvent les plus utilisés (ex. : Cessna, Piper).
- Types d’enquêtes : majoritairement des investigations complètes ou préliminaires.

 Interprétations

- Les accidents ne diminuent pas nécessairement avec le temps malgré les avancées technologiques.
- La concentration géographique (notamment aux USA) peut être liée à la fréquence des vols ou à la rigueur des enregistrements.
- Le type de constructeur n’indique pas directement une dangerosité, mais plutôt une fréquence d’usage.
- La majorité des incidents ne causent pas de morts, ce qui reflète l’efficacité des mesures de sécurité modernes.

 Conclusion

Cette analyse permet d’avoir une vue d’ensemble des dynamiques liées aux accidents aériens dans le jeu de données. Pour aller plus loin, des modèles prédictifs ou des corrélations approfondies pourraient être mis en œuvre pour identifier les causes principales d’accidents selon le type de vol, les conditions météo ou d’autres variables.

Étape suivante possible : création d’un tableau de bord interactif dans Power BI, ou mise en ligne du projet sur GitHub.


In [22]:
# Étape 1 : Supprimer les colonnes inutiles (exemple : celles qui sont vides ou non pertinentes)
colonnes_a_supprimer = ['Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 'Broad.Phase.of.Flight', 'Weather.Condition']
aviation_df.drop(columns=colonnes_a_supprimer, inplace=True, errors='ignore')

# Étape 2 : Supprimer les lignes trop incomplètes
aviation_df.dropna(thresh=5, inplace=True)  # garde les lignes avec au moins 5 valeurs non nulles

# Étape 3 : Convertir les dates au bon format
aviation_df['Event.Date'] = pd.to_datetime(aviation_df['Event.Date'], errors='coerce')

# Étape 4 : Supprimer les doublons (si nécessaire)
aviation_df.drop_duplicates(inplace=True)

# Étape 5 : Exporter vers un fichier CSV propre
aviation_df.to_csv("AviationData_cleaned.csv", index=False)

print("✅ Fichier exporté avec succès : AviationData_cleaned.csv")


✅ Fichier exporté avec succès : AviationData_cleaned.csv
